In [ ]:
! pip install pydicom
import cv2
import numpy
import glob
import os
import pydicom # pydicom is using the gdcm package for decompression
import matplotlib.pyplot as plt
import pandas as pd
import shutil
from scipy import ndimage

In [ ]:
folders = sorted(glob.glob('Your Path Here//*'))
data = pd.read_csv('Your Path Here/SeriesNumbers_modified.csv', header = None)
data.columns =['PatientID', 'SeriesNumber']

for i, path in enumerate(folders):
    study = os.path.basename(os.path.normpath(path)) 
    study = study.replace(" ", "")
    print(study)
    unsortedList = []
    dst = os.path.join(path, 'sorted')
    for file in os.listdir(path):
        d = os.path.join(path, file)
        if os.path.isdir(d):
           for root, dirs, files in os.walk(path):
            for file in files: 
                ds = pydicom.read_file(root + '/' + file, force=True)
                if ds.SeriesNumber == data.loc[data['PatientID'] == study, 'SeriesNumber'].iloc[0]:
                   unsortedList.append(os.path.join(root, file))
        else:
            ds = pydicom.read_file(path + '/' + file, force=True)
            if ds.SeriesNumber == data.loc[data['PatientID'] == study, 'SeriesNumber'].iloc[0]:
                unsortedList.append(os.path.join(path, file))
        
                  

    for dicom_loc in unsortedList:
        # read the file
        ds = pydicom.read_file(dicom_loc, force=True)
        # generate new, standardized file name
        instanceNumber = str(ds.get("InstanceNumber","0"))
        fileName = instanceNumber + ".dcm"
        # uncompress files (using the gdcm package)
        try:
            ds.decompress()
        except:
            print('an instance in file %s" could not be decompressed. exiting.' % (seriesDescription))
      
        # save sorted files
        if not os.path.exists(dst):
            os.makedirs(dst)
          
        ds.save_as(os.path.join(dst, fileName))

    print('done.')

In [ ]:
folders = sorted(glob.glob('Your Path Here//*'))
for i, path in enumerate(folders):
    dst = os.path.join(path, 'sorted')
    if os.path.exists(dst):
       shutil.rmtree(dst)
    